In [ ]:
from functools import partial
import numpy as np

def _obj_wrapper(func, args, kwargs, x):
    return func(x, *args, **kwargs)

def _is_feasible_wrapper(func, x):
    return np.all(func(x)>=0)

def _cons_none_wrapper(x):
    return np.array([0])

def _cons_ieqcons_wrapper(ieqcons, args, kwargs, x):
    return np.array([y(x, *args, **kwargs) for y in ieqcons])

def _cons_f_ieqcons_wrapper(f_ieqcons, args, kwargs, x):
    return np.array(f_ieqcons(x, *args, **kwargs))
    
def pso(func, lb, ub, ieqcons=[], f_ieqcons=None, args=(), kwargs={}, 
        swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=100, 
        minstep=1e-8, minfunc=1e-8, debug=False, processes=1,
        particle_output=False):
    """
    Perform a particle swarm optimization (PSO)
   
    Parameters
    ==========
    func : function
        The function to be minimized
    lb : array
        The lower bounds of the design variable(s)
    ub : array
        The upper bounds of the design variable(s)
   
    Optional
    ========
    ieqcons : list
        A list of functions of length n such that ieqcons[j](x,*args) >= 0.0 in 
        a successfully optimized problem (Default: [])
    f_ieqcons : function
        Returns a 1-D array in which each element must be greater or equal 
        to 0.0 in a successfully optimized problem. If f_ieqcons is specified, 
        ieqcons is ignored (Default: None)
    args : tuple
        Additional arguments passed to objective and constraint functions
        (Default: empty tuple)
    kwargs : dict
        Additional keyword arguments passed to objective and constraint 
        functions (Default: empty dict)
    swarmsize : int
        The number of particles in the swarm (Default: 100)
    omega : scalar
        Particle velocity scaling factor (Default: 0.5)
    phip : scalar
        Scaling factor to search away from the particle's best known position
        (Default: 0.5)
    phig : scalar
        Scaling factor to search away from the swarm's best known position
        (Default: 0.5)
    maxiter : int
        The maximum number of iterations for the swarm to search (Default: 100)
    minstep : scalar
        The minimum stepsize of swarm's best position before the search
        terminates (Default: 1e-8)
    minfunc : scalar
        The minimum change of swarm's best objective value before the search
        terminates (Default: 1e-8)
    debug : boolean
        If True, progress statements will be displayed every iteration
        (Default: False)
    processes : int
        The number of processes to use to evaluate objective function and 
        constraints (default: 1)
    particle_output : boolean
        Whether to include the best per-particle position and the objective
        values at those.
   
    Returns
    =======
    g : array
        The swarm's best known position (optimal design)
    f : scalar
        The objective value at ``g``
    p : array
        The best known position per particle
    pf: arrray
        The objective values at each position in p
   
    """
   
    assert len(lb)==len(ub), 'Lower- and upper-bounds must be the same length'
    assert hasattr(func, '__call__'), 'Invalid function handle'
    lb = np.array(lb)
    ub = np.array(ub)
    assert np.all(ub>lb), 'All upper-bound values must be greater than lower-bound values'
   
    vhigh = np.abs(ub - lb)
    vlow = -vhigh

    # Initialize objective function
    obj = partial(_obj_wrapper, func, args, kwargs)
    
    # Check for constraint function(s) #########################################
    if f_ieqcons is None:
        if not len(ieqcons):
            if debug:
                print('No constraints given.')
            cons = _cons_none_wrapper
        else:
            if debug:
                print('Converting ieqcons to a single constraint function')
            cons = partial(_cons_ieqcons_wrapper, ieqcons, args, kwargs)
    else:
        if debug:
            print('Single constraint function given in f_ieqcons')
        cons = partial(_cons_f_ieqcons_wrapper, f_ieqcons, args, kwargs)
    is_feasible = partial(_is_feasible_wrapper, cons)

    # Initialize the multiprocessing module if necessary
    if processes > 1:
        import multiprocessing
        mp_pool = multiprocessing.Pool(processes)
        
    # Initialize the particle swarm ############################################
    S = swarmsize
    D = len(lb)  # the number of dimensions each particle has
    x = np.random.rand(S, D)  # particle positions
    v = np.zeros_like(x)  # particle velocities
    p = np.zeros_like(x)  # best particle positions
    fx = np.zeros(S)  # current particle function values
    fs = np.zeros(S, dtype=bool)  # feasibility of each particle
    fp = np.ones(S)*np.inf  # best particle function values
    g = []  # best swarm position
    fg = np.inf  # best swarm position starting value
    
    # Initialize the particle's position
    x = lb + x*(ub - lb)

    # Calculate objective and constraints for each particle
    if processes > 1:
        fx = np.array(mp_pool.map(obj, x))
        fs = np.array(mp_pool.map(is_feasible, x))
    else:
        for i in range(S):
            fx[i] = obj(x[i, :])
            fs[i] = is_feasible(x[i, :])
       
    # Store particle's best position (if constraints are satisfied)
    i_update = np.logical_and((fx < fp), fs)
    p[i_update, :] = x[i_update, :].copy()
    fp[i_update] = fx[i_update]

    # Update swarm's best position
    i_min = np.argmin(fp)
    if fp[i_min] < fg:
        fg = fp[i_min]
        g = p[i_min, :].copy()
    else:
        # At the start, there may not be any feasible starting point, so just
        # give it a temporary "best" point since it's likely to change
        g = x[0, :].copy()
       
    # Initialize the particle's velocity
    v = vlow + np.random.rand(S, D)*(vhigh - vlow)
       
    # Iterate until termination criterion met ##################################
    it = 1
    while it <= maxiter:
        rp = np.random.uniform(size=(S, D))
        rg = np.random.uniform(size=(S, D))

        # Update the particles velocities
        v = omega*v + phip*rp*(p - x) + phig*rg*(g - x)
        # Update the particles' positions
        x = x + v
        # Correct for bound violations
        maskl = x < lb
        masku = x > ub
        x = x*(~np.logical_or(maskl, masku)) + lb*maskl + ub*masku

        # Update objectives and constraints
        if processes > 1:
            fx = np.array(mp_pool.map(obj, x))
            fs = np.array(mp_pool.map(is_feasible, x))
        else:
            for i in range(S):
                fx[i] = obj(x[i, :])
                fs[i] = is_feasible(x[i, :])

        # Store particle's best position (if constraints are satisfied)
        i_update = np.logical_and((fx < fp), fs)
        p[i_update, :] = x[i_update, :].copy()
        fp[i_update] = fx[i_update]

        # Compare swarm's best position with global best position
        i_min = np.argmin(fp)
        if fp[i_min] < fg:
            if debug:
                print('New best for swarm at iteration {:}: {:} {:}'\
                    .format(it, p[i_min, :], fp[i_min]))

            p_min = p[i_min, :].copy()
            stepsize = np.sqrt(np.sum((g - p_min)**2))

            if np.abs(fg - fp[i_min]) <= minfunc:
                print('Stopping search: Swarm best objective change less than {:}'\
                    .format(minfunc))
                if particle_output:
                    return p_min, fp[i_min], p, fp
                else:
                    return p_min, fp[i_min]
            elif stepsize <= minstep:
                print('Stopping search: Swarm best position change less than {:}'\
                    .format(minstep))
                if particle_output:
                    return p_min, fp[i_min], p, fp
                else:
                    return p_min, fp[i_min]
            else:
                g = p_min.copy()
                fg = fp[i_min]

        if debug:
            print('Best after iteration {:}: {:} {:}'.format(it, g, fg))
        it += 1

    print('Stopping search: maximum iterations reached --> {:}'.format(maxiter))
    
    if not is_feasible(g):
        print("However, the optimization couldn't find a feasible design. Sorry")
    if particle_output:
        return g, fg, p, fp
    else:
        return g, fg
 

In [ ]:

def objective(x):
    return x[0]**2 + x[1]**2  # пример функции для минимизации

lb = [-10, -10]  # нижние границы
ub = [10, 10]    # верхние границы

xopt, fopt = pso(objective, lb, ub, swarmsize=30, maxiter=100)
print(f"Оптимальное решение: {xopt}, значение функции: {fopt}")


In [ ]:
import numpy as np
import EFIE as solve
light_speed, mu0, eps0 = 299792458., 4*np.pi*1e-7, 8.854e-12

frequency = 1e6
omega, wavelength = 2 * np.pi * frequency, light_speed / frequency
incident_field, radius = 220, 3.175e-3
delta_r = wavelength / 200

class yagi:
    def __init__(self, position, angle, length, source_position, radius):
        self.position = position
        self.angle = angle
        self.length = length
        self.source_position = source_position
        self.radius = radius

class tree:
    def __init__(self, its, phi, length, f, radius, field):
        self.its = its
        self.phi = phi
        self.length = length
        self.f = f
        self.radius = radius
        self.field = field

structure_type = 'tree'
basis_functions = 'triangle'
its = 3
length = wavelength/10
f = 1
radius = 3.175e-3
field = incident_field

def objective(phi):
    tree_test = tree(its,phi,length,f,radius,field)
    I,R = solve.calc_current_amplitudes(structure_type, basis_functions, tree_test, frequency, delta_r)
    E,angles = solve.calc_field_pattern(0,0,basis_functions,structure_type,tree_test,I,R,delta_r,frequency)
    E = E / np.max(E)
    aim_func = np.zeros(len(angles))
    for i in range(len(angles)):
        aim_func[i] = max(0, np.sin(angles[i])**7)
    fit = 1 - np.dot(E,aim_func) / np.linalg.norm(E) / np.linalg.norm(aim_func)
    print(fit)
    return fit

lb = np.full(2 * (2 ** its - 1), 0.0)
ub = np.full(2 * (2 ** its - 1), np.pi/2)

xopt, fopt = pso(objective, lb, ub, swarmsize=10, maxiter=20)
print(f"Оптимальное решение: {xopt}, значение функции: {fopt}")


In [ ]:
structure_type = 'tree'
basis_functions = 'triangle'
its = 3
length = wavelength/2
f = 1/np.sqrt(2)
radius = 3.175e-3
field = incident_field
phi = xopt
tree_opt = tree(its,phi,length,f,radius,field)
I,R = solve.calc_current_amplitudes(structure_type, basis_functions, tree_opt, frequency, delta_r)
E,angles = solve.calc_field_pattern(0,0,basis_functions,structure_type,tree_opt,I,R,delta_r,frequency)

In [ ]:
import geometry as gm
gm.plot_distribution(I,R,frequency)

In [ ]:
import matplotlib.pyplot as plt 
aim_func = np.zeros(len(angles))
for i in range(len(angles)):
    aim_func[i] = max(0, np.sin(angles[i])**7)
E = E / np.max(E)
plt.polar(angles,aim_func, c = 'r')
plt.polar(angles, E, c = 'darkblue')

In [ ]:
import numpy as np
import EFIE as solve

light_speed, mu0, eps0 = 299792458., 4 * np.pi * 1e-7, 8.854e-12

frequency = 1e6 * 146
omega, wavelength = 2 * np.pi * frequency, light_speed / frequency
incident_field, radius = 220, 3.175e-3
delta_r = wavelength / 100

structure_type = 'symetric_tree'
class symetric_tree:
    def __init__(self, its, phi, length, f, radius, field):
        self.its = its
        self.phi = phi
        self.length = length
        self.f = f
        self.radius = radius
        self.field = field
basis_functions = 'triangle'

its = 3
length = wavelength / 10
f = 1
radius = 3.175e-3
field = incident_field

all_results = {
    'phi': [], 
    'I': [],
    'R': [],
    'E': [],
    'angles': [],
    'fit': []
}

def objective(phi):
    tree_test = symetric_tree(its, phi[0][0], length, f, radius, field)
    I, R, _, _, _, _ = solve.calc_current_amplitudes(structure_type, basis_functions, tree_test, frequency, delta_r)
    E, angles = solve.calc_field_pattern(0, 0, 1e2,basis_functions, structure_type, tree_test, I, R, delta_r, frequency)
    E = E / np.max(E)
    aim_func = np.zeros(len(angles))
    for i in range(len(angles)):
        aim_func[i] = max(0, np.sin(angles[i]) ** 7)
        E[i] = E[i] / np.max(E)
    fit = 1 - np.dot(E, aim_func) / np.linalg.norm(E) / np.linalg.norm(aim_func)

    all_results['phi'].append(np.copy(phi))
    all_results['I'].append(np.copy(I))
    all_results['R'].append(np.copy(R))
    all_results['E'].append(np.copy(E))
    all_results['angles'].append(np.copy(angles))
    all_results['fit'].append(fit)

    return fit

lb = np.full(1, 0.0)
ub = np.full(1, np.pi / 2)

import pyswarms as ps

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
n_particles = 10
dimensions = 1
bounds = (lb, ub)

optimizer = ps.single.GlobalBestPSO(
    n_particles=n_particles,
    dimensions=dimensions,
    options=options,
    bounds=bounds
)

best_cost, best_pos = optimizer.optimize(objective, iters=30)
print(f"Best position: {best_pos}")
print(f"Best cost: {best_cost}")

np.savez(f'all_results1.npz', **all_results, dtype = object)


In [ ]:
import numpy as np
import EFIE as solve

light_speed, mu0, eps0 = 299792458., 4 * np.pi * 1e-7, 8.854e-12

frequency = 1e6 * 146
omega, wavelength = 2 * np.pi * frequency, light_speed / frequency
incident_field, radius = 220, 3.175e-3
delta_r = wavelength / 100

class tree:
    def __init__(self, its, phi, length, f, radius, field):
        self.its = its
        self.phi = phi
        self.length = length
        self.f = f
        self.radius = radius
        self.field = field

structure_type = 'tree'
basis_functions = 'triangle'
its = 3
length = wavelength / 10
f = 1
radius = 3.175e-3
field = incident_field

all_results = {
    'phi': [], 
    'I': [],
    'R': [],
    'E': [],
    'angles': [],
    'fit': []
}

def objective(phi):
    tree_test = tree(its, phi[0], length, f, radius, field)
    I, R, _, _, _, _ = solve.calc_current_amplitudes(structure_type, basis_functions, tree_test, frequency, delta_r)
    E, angles = solve.calc_field_pattern(0, 0, 1e2,basis_functions, structure_type, tree_test, I, R, delta_r, frequency)
    E = E / np.max(E)
    aim_func = np.zeros(len(angles))
    for i in range(len(angles)):
        aim_func[i] = max(0, np.sin(angles[i]) ** 7)
        E[i] = E[i] / np.max(E)
    fit = 1 - np.dot(E, aim_func) / np.linalg.norm(E) / np.linalg.norm(aim_func)

    all_results['phi'].append(np.copy(phi))
    all_results['I'].append(np.copy(I))
    all_results['R'].append(np.copy(R))
    all_results['E'].append(np.copy(E))
    all_results['angles'].append(np.copy(angles))
    all_results['fit'].append(fit)

    return fit

lb = np.full(2 * (2 ** its - 1), 0.0)
ub = np.full(2 * (2 ** its - 1), np.pi / 2)

import pyswarms as ps

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
n_particles = 5
dimensions = 2 * (2 ** its - 1)
bounds = (lb, ub)

optimizer = ps.single.GlobalBestPSO(
    n_particles=n_particles,
    dimensions=dimensions,
    options=options,
    bounds=bounds
)

best_cost, best_pos = optimizer.optimize(objective, iters=30)
print(f"Best position: {best_pos}")
print(f"Best cost: {best_cost}")

np.savez(f'all_results1.npz', **all_results, dtype = object)
